# Solving a two-asset portfolio optimization model with SciPy

A portfolio optimization problem can be presented by the following optimization problem. This is a nonlinear (quadratic) optimization problem:
$$
\begin{equation}
\begin{array}
\displaystyle \min_{x_1,x_2} & \sigma_1^2 \cdot x_1^2 + \sigma_2^2 \cdot x_2^2 + 2 \cdot x_1 \cdot x_2 \cdot \sigma_{12} \\
\textrm{s.t.}                & x_1 + x_2 = 1 \\
                             & r_1 \cdot x_1 + r_2 \cdot x_2 \geq r_\text{Target} \\
                             & x_1 \geq 0, x_2 \geq 0 \\
\end{array}
\end{equation}
$$

Here $\sigma_1^2$ and $\sigma_2^2$ are the variances of the returns of assets #1 and #2, $x_1$ and $x_2$ are the weights of the assets in the portfolio, $r_1$ and $r_2$ are the expected asset returns, $\sigma_{12}$ is the covariance between the returns of the two assets, and $r_\text{Target}$ is the target expected return of the portfolio.


We will use the following input data: $\sigma_1=0.1$, $\sigma_2=0.13$, $\sigma_{12}=0.001$, $r_1=0.15$, $r_2=0.2$ and $r_\text{Target}=0.18$. **Note** that we here have specified the standard deviations $\sigma_1$ and $\sigma_2$,

With these numbers the problem is the following:

$$
\begin{equation}
\begin{array}
\displaystyle \min_{x_1,x_2} & 0.1^2 \cdot x_1^2 + 0.13^2 \cdot x_2^2 + 2 \cdot x_1 \cdot x_2 \cdot 0.001 \\
\textrm{s.t.}                & x_1 + x_2 = 1 \\
                             & 0.15 \cdot x_1 + 0.20 \cdot x_2 \geq 0.18 \\
                             & x_1 \geq 0, x_2 \geq 0 \\
\end{array}
\end{equation}
$$

We will use the `SciPy` (Scientific Python) library. And from this library we will use the `optimize` module, and more particularly the `minimize` submodul/function. If you do not have the `SciPy` package from before, then you can install it by running the following code:'
```
! pip install scipy
```

Let us now import the relevant modules from this library:

In [1]:
from scipy.optimize import minimize
import numpy as np

Let us here define the parameters.

In [2]:
p = [0.10, 0.13, 0.001, 0.15, 0.20, 0.18, 1.0]

Let us also have a look what the datatype is:

In [3]:
type(p)

list

Now we will create our model, which consists of the objective function and two constraints.

In [4]:
def objective(x, p):
    return p[0]**2*x[0]**2 + p[1]**2*x[1]**2  + 2*p[2]*x[0]*x[1]

The budget (total weights) constraint:

In [5]:
def leftside_1(x,p):
    return x[0]+x[1]-p[6]

The target return constraint:

In [6]:
def leftside_2(x,p):
    return p[3]*x[0] + p[4]*x[1] - p[5]

In SciPy the constraints need to be defined in the form of a dictionary.

**Note** that the inequality cosntraint is of type **$ \geq $** in SciPy.

The following construction is slightly non-intuitive, because we have to write `'args': (p,)` insteead of `'args': p`. However, SciPy will complain that it receives two many parameters, if we do not explicitely instruct that args is a tuple.
Even if we had defined `p` as a tuple in the first place, like: `p = (1, 5, 7, 9, 3)`, it would not work.

In [7]:
constraint_1=({'type': 'eq', 'fun': leftside_1, 'args': (p,)})

In [8]:
constraint_2=({'type': 'ineq', 'fun': leftside_2, 'args': (p,)})

In [9]:
print((p,))

([0.1, 0.13, 0.001, 0.15, 0.2, 0.18, 1.0],)


In [10]:
constraints = [constraint_1, constraint_2]

Let us just run these function and see what they produce:

In [11]:
x0 = [1, 0]

In [12]:
print("the objective value is " + str(objective(x0,p)))
print("the value of constraint 1 is " + str(leftside_1(x0,p)))
print("the value of constraint 2 is " + str(leftside_2(x0,p)))

the objective value is 0.010000000000000002
the value of constraint 1 is 0.0
the value of constraint 2 is -0.03


I make a numpy array from the tuple p.

In [13]:
p0=np.array(p)

In [14]:
results=minimize(objective, x0, args=p, method='SLSQP', constraints=constraints)

In [15]:
print(results)

     fun: 0.008164000159323217
     jac: array([0.0092 , 0.02108])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.39999999, 0.60000001])


Let us assign the solution and calculate the portfolio return.

In [16]:
x = results['x']

In [17]:
print("The optimal weights are ", x)
PortfReturn=x[0]*p[3]+x[1]*p[4]
print("The expected portfolio return is: ",PortfReturn) 
print("The portfolio variance is: ", objective(x, p))

The optimal weights are  [0.39999999 0.60000001]
The expected portfolio return is:  0.18000000067055225
The portfolio variance is:  0.008164000159323217


**THE END**